In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

In [15]:
data_df_fine = pd.read_csv('Genelec8020c_1x1_64442_IR_front_pole.csv',
                        delimiter=',',
                        decimal='.',
                        nrows=100
                    )
data_df_fine

,P000T000,6.3958492,-3.5932083,8.3997803,-2.7310836,5.5528570,-2.5485539,7.8546187,-1.8450409,5.6827591,...,-2.4282285,8.4949489,-1.8001793,4.5874018,-5.3256966,3.2223002,-5.4540911,5.5704187,-4.5273033,3.8711031
0,P000T001,6.259440,-3.924950,8.273696,-1.148981,5.497420,-3.028312,7.277060,-1.365671,5.153759,...,-3.034938,8.146634,-1.485814,4.794820,-6.020558,3.740175,-5.629463,4.934990,-3.480578,3.589432
1,P001T001,6.276859,-3.932021,8.281501,-1.148011,5.513724,-3.038702,7.288230,-1.385072,5.174320,...,-3.033681,8.141576,-1.486179,4.795979,-6.031941,3.754395,-5.627184,4.930251,-3.491780,3.587884
2,P002T001,6.294568,-3.939223,8.289683,-1.147755,5.530132,-3.049146,7.299804,-1.404847,5.195335,...,-3.032673,8.137020,-1.486947,4.797362,-6.043428,3.768829,-5.625270,4.925987,-3.503557,3.586624
3,P003T001,6.312545,-3.946544,8.298226,-1.148198,5.546624,-3.059630,7.311762,-1.424977,5.216788,...,-3.031909,8.132955,-1.488105,4.798953,-6.055001,3.783462,-5.623711,4.922186,-3.515887,3.585637
4,P004T001,6.330773,-3.953970,8.307117,-1.149325,5.563176,-3.070136,7.324082,-1.445440,5.238657,...,-3.031381,8.129370,-1.489636,4.800735,-6.066643,3.798278,-5.622498,4.918834,-3.528747,3.584910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,P095T001,6.082740,-2.966893,7.771925,-1.638527,5.055264,-1.927217,7.045569,-1.627582,5.763628,...,-1.660007,7.260682,-0.805967,3.437796,-4.693503,3.135187,-4.489978,4.191269,-3.823558,2.713955
96,P096T001,6.038494,-2.923294,7.727551,-1.626623,5.018957,-1.880499,7.010055,-1.592351,5.729191,...,-1.608819,7.222204,-0.771621,3.393745,-4.634873,3.082216,-4.440366,4.158278,-3.794344,2.683673
97,P097T001,5.993521,-2.879177,7.682382,-1.614266,4.982328,-1.833359,6.974107,-1.556583,5.694025,...,-1.556897,7.183096,-0.736783,3.349379,-4.575626,3.028328,-4.389847,4.124685,-3.764448,2.653089
98,P098T001,5.947854,-2.834570,7.636446,-1.601471,4.945407,-1.785827,6.937755,-1.520312,5.658162,...,-1.504271,7.143381,-0.701476,3.304732,-4.515805,2.973557,-4.338447,4.090509,-3.733894,2.622225


In [16]:

def to_complex_safe(val):
    """
    Converts a string from the CSV to a complex number.
    - Handles 'NaN - NaNi'
    - Replaces 'i' with 'j' for Python
    - Removes spaces to allow complex() to parse correctly
    """
    if pd.isna(val):
        return np.nan
    if isinstance(val, str):
        # Trim leading/trailing whitespace
        val = val.strip()
        
        # Handle the specific NaN format
        if val == 'NaN - NaNi':
            return np.nan
            
        # Replace 'i' with 'j' for Python
        val = val.replace('i', 'j')
        
        # Remove ALL spaces (e.g., "4.4 + 16.0j" -> "4.4+16.0j")
        val = val.replace(' ', '') 
        
        try:
            # Now complex() should work
            return complex(val)
        except ValueError:
            return np.nan
    
    return val

# Use the new filename provided by the user
filename = 'Genelec8020_1x1_64442_MPS_front_pole.csv'

# 1. Read the CSV
data_df = pd.read_csv(filename,
                        delimiter=',',
                        decimal='.',
                        header=0,        # The first row is the header
                        index_col=0,     # Use the first column as the index
                        nrows = 64442,
                        skipinitialspace=True)

# 2. Clean up potential extra column
data_df = data_df.dropna(axis=1, how='all')

# 3. Convert all data to complex numbers
data_df = data_df.map(to_complex_safe)

# 4. Convert column headers to numbers
data_df.columns = pd.to_numeric(data_df.columns, errors='coerce')

# 5. Drop any columns that might have failed header conversion
data_df = data_df.loc[:, data_df.columns.notna()]

# 6. Drop any rows that are all NaN (if any exist)
data_df = data_df.dropna(axis=0, how='all')
data_df = data_df.drop(columns=[25.0])

# 7. Display the results
print(f"Successfully loaded '{filename}'. Index name is: '{data_df.index.name}' and there are {len(data_df)} rows and {len(data_df.columns)} columns")
print("\nDataFrame Head:")
data_df.head(2)
    

Successfully loaded 'Genelec8020_1x1_64442_MPS_front_pole.csv'. Index name is: 'f in Hz' and there are 64442 rows and 30 columns

DataFrame Head:


,20.0,31.5,40.0,50.0,63.0,80.0,100.0,125.0,160.0,200.0,...,2500.0,3150.0,4000.0,5000.0,6300.0,8000.0,10000.0,12500.0,16000.0,20000.0
f in Hz,,,,,,,,,,,,,,,,,,,,,
P000T000,4.407277+16.052855j,-81.871808-72.987188j,-159.751517+233.789089j,590.886252+1505.674253j,2774.259344+1140.102858j,2731.612138-1772.992986j,-267.186504-3226.556713j,-3448.125272-1234.293769j,-2319.411850+2747.499086j,1963.604172+3177.852308j,...,-673.856499-4188.676059j,-2988.066596-3266.020280j,3548.615973-1168.721676j,1043.457641-3343.308653j,3605.752800- 501.319863j,4059.742121- 192.206141j,2314.717207-3069.279796j,-2702.175315+1916.673539j,-3492.623983- 496.748508j,2088.190897-2822.657351j
P000T001,4.430241+16.059229j,-81.943978-72.831609j,-159.332343+234.134272j,594.626282+1504.190695j,2777.432212+1132.295718j,2725.521464-1782.337673j,-281.025450-3225.385274j,-3454.867655-1215.300881j,-2299.619988+2764.092762j,1993.854399+3158.803643j,...,-1253.037596-4066.882521j,-3392.252023-2926.437854j,3355.834278-1693.913422j,347.724024-3553.664219j,3379.041027-1389.218703j,3861.971367-1366.970620j,1153.738850-3633.490339j,-1584.183970+2892.463012j,-3110.945081+1559.911455j,-244.743085-3456.140370j


In [171]:
# Create a dataframe with normalized magnitudes
print("\nCreating normalized magnitudes dataframe...")

# Initialize empty dataframe to store normalized magnitudes
normalized_df = pd.DataFrame(index=data_df.index, columns=data_df.columns)

# Calculate normalized magnitudes for each frequency
for col in data_df.columns:
    # Get on-axis magnitude for this frequency (handle duplicates)
    on_axis_value = data_df.loc['P000T000', col]
    if hasattr(on_axis_value, 'iloc'):  # Multiple values returned (duplicates)
        on_axis_value = on_axis_value.iloc[0]
    on_axis_magnitude = np.abs(on_axis_value)
    
    # Normalize all magnitudes for this frequency
    normalized_df[col] = np.abs(data_df[col].values) / on_axis_magnitude

# Remove duplicate rows (if they exist due to the duplicate data issue)
normalized_df = normalized_df[~normalized_df.index.duplicated(keep='first')]

print(f"Normalized dataframe created with {len(normalized_df)} positions and {len(normalized_df.columns)} frequencies")
print(f"\nDataframe shape: {normalized_df.shape}")

# Save to CSV
output_csv = 'Genelec8020_normalized_magnitudes.csv'
normalized_df.to_csv(output_csv)
print(f"\nSaved normalized magnitudes to '{output_csv}'")


Creating normalized magnitudes dataframe...
Normalized dataframe created with 64442 positions and 30 frequencies

Dataframe shape: (64442, 30)

Saved normalized magnitudes to 'Genelec8020_normalized_magnitudes.csv'


In [18]:
# Calculate global colormap range for normalized dB values
print("Calculating global normalization and dB range...")

# We'll calculate the global range by processing each frequency column
all_normalized_db = []

for col in data_df.columns:
    # Get on-axis magnitude for this frequency (take first occurrence if duplicates exist)
    on_axis_value = data_df.loc['P000T000', col]
    if hasattr(on_axis_value, 'iloc'):  # Multiple values returned (duplicates)
        on_axis_value = on_axis_value.iloc[0]
    on_axis_magnitude = np.abs(on_axis_value)
    
    # Normalize all magnitudes for this frequency
    normalized_magnitudes = np.abs(data_df[col].values) / on_axis_magnitude
    # Convert to dB
    normalized_db = 20 * np.log10(normalized_magnitudes + 1e-12)
    all_normalized_db.extend(normalized_db[~np.isnan(normalized_db)])

# Set global colormap range for dB values
actual_vmin_db = np.percentile(all_normalized_db, 1)
actual_vmax_db = np.percentile(all_normalized_db, 99)
print(f"Actual dB range (1st to 99th percentile): {actual_vmin_db:.2f} dB to {actual_vmax_db:.2f} dB")

# Use a wider range for better color contrast in directivity plots
global_vmin_db = -10  # Typical directivity range
global_vmax_db = 0    # On-axis should be 0 dB
print(f"Colormap dB range: {global_vmin_db:.2f} dB to {global_vmax_db:.2f} dB")

for plot_frequency in data_df.columns.to_list()[:]:
        
        if plot_frequency not in data_df.columns:
                print(f"Error: Frequency {plot_frequency} Hz not found in the columns.")
                print(f"Available frequencies are: {data_df.columns.tolist()}")
        else:
                print(f"Extracting data for {plot_frequency} Hz...")
                
                # Select the column for this frequency
                data_plot_frequency = data_df[plot_frequency]
                
                # Get on-axis magnitude for normalization (handle duplicates)
                try:
                        on_axis_value = data_plot_frequency.loc['P000T000']
                        if hasattr(on_axis_value, 'iloc'):  # Multiple values returned
                                on_axis_value = on_axis_value.iloc[0]
                        on_axis_magnitude = np.abs(on_axis_value)
                except KeyError:
                        print("Error: 'P000T000' not found in index. Using the first value as on-axis.")
                        on_axis_magnitude = np.abs(data_plot_frequency.iloc[0])

                # Normalize and convert to dB
                normalized_magnitudes = np.abs(data_plot_frequency) / on_axis_magnitude
                normalized_magnitudes_db = 20 * np.log10(normalized_magnitudes + 1e-12)
                
                # Get the position strings (PXXXTXXX)
                positions = data_plot_frequency.index
                
                # Parse Phi (φ) and Theta (θ) from the index strings
                phi_deg = positions.str.extract(r'P(\d+)T\d+')[0].astype(float)
                theta_deg = positions.str.extract(r'P\d+T(\d+)')[0].astype(float)
                
                # --- Convert Spherical to Cartesian Coordinates ---
                phi_rad = np.deg2rad(phi_deg)
                theta_rad = np.deg2rad(theta_deg)
                
                r = 1 
                x = r * np.cos(theta_rad)
                y = r * np.sin(theta_rad) * np.cos(phi_rad)
                z = r * np.sin(theta_rad) * np.sin(phi_rad)
                
                # --- Create the 3D Plot ---
                print("Generating 3D plot...")
                fig = plt.figure(figsize=(10, 10))
                ax = fig.add_subplot(111, projection='3d')
                
                # Disable automatic depth sorting so we can control zorder manually
                ax.computed_zorder = False

                # Create scatter plot with normalized dB colormap (alpha=0.5 for transparency)
                scatter = ax.scatter(x, y, z, c=normalized_magnitudes_db, cmap='RdBu_r', s=50, 
                                   vmin=actual_vmin_db, vmax=global_vmax_db, zorder=1)
                
                # Add coordinate system arrows (quivers) - higher zorder so they're on top
                ax.quiver(0, 0, 0, 1.5, 0, 0, color='red', arrow_length_ratio=0.05, linewidth=3, label='X (Front)', zorder=10)
                ax.quiver(0, 0, 0, 0, 1.5, 0, color='green', arrow_length_ratio=0.05, linewidth=3, label='Y', zorder=10)
                ax.quiver(0, 0, 0, 0, 0, 1.5, color='blue', arrow_length_ratio=0.05, linewidth=3, label='Z', zorder=10)
                
                # Add a color bar to show the dB scale
                cbar = fig.colorbar(scatter, ax=ax, label='Magnitude (dB re. on-axis)', shrink=0.6)
                
                # Set titles and labels
                ax.set_title(f'Normalized Directivity at {plot_frequency} Hz')
                ax.set_xlabel('X-axis (On-Axis)')
                ax.set_ylabel('Y-axis')
                ax.set_zlabel('Z-axis')
                
                # Set axis limits
                ax.set_xlim([-1.5, 1.5])
                ax.set_ylim([-1.5, 1.5])
                ax.set_zlim([-1.5, 1.5])
                
                # IMPORTANT: Force equal aspect ratio for 3D plot
                ax.set_box_aspect([1, 1, 1])

                ax.set_xticks(np.arange(-1, 1.5, 0.5))
                ax.set_yticks(np.arange(-1, 1.5, 0.5))
                ax.set_zticks(np.arange(-1, 1.5, 0.5))
                
                ax.legend(loc='upper left')
                
                # Save the plot
                output_filename = f"spherical_plots/sperical_plot_{str(plot_frequency).replace('.','_')}Hz.png"
                plt.savefig(output_filename, dpi=150, bbox_inches='tight')
                plt.close(fig) # Close the figure to free up memory
                
                print(f"Successfully saved plot to '{output_filename}'")

Calculating global normalization and dB range...
Actual dB range (1st to 99th percentile): -29.25 dB to 0.42 dB
Colormap dB range: -10.00 dB to 0.00 dB
Extracting data for 20.0 Hz...
Generating 3D plot...
Successfully saved plot to 'spherical_plots/sperical_plot_20_0Hz.png'
Extracting data for 31.5 Hz...
Generating 3D plot...
Successfully saved plot to 'spherical_plots/sperical_plot_31_5Hz.png'
Extracting data for 40.0 Hz...
Generating 3D plot...
Successfully saved plot to 'spherical_plots/sperical_plot_40_0Hz.png'
Extracting data for 50.0 Hz...
Generating 3D plot...
Successfully saved plot to 'spherical_plots/sperical_plot_50_0Hz.png'
Extracting data for 63.0 Hz...
Generating 3D plot...
Successfully saved plot to 'spherical_plots/sperical_plot_63_0Hz.png'
Extracting data for 80.0 Hz...
Generating 3D plot...
Successfully saved plot to 'spherical_plots/sperical_plot_80_0Hz.png'
Extracting data for 100.0 Hz...
Generating 3D plot...
Successfully saved plot to 'spherical_plots/sperical_plo